# Accounting Logic

In [78]:
import random
import numpy as np

random.seed(0)

In [79]:
assets = ['asset1', 'asset2']
data_source = (lambda assets: ({asset: random.random() for asset in assets} for i in range(1000000)))(assets)

In [80]:
global current_prices 
current_prices = next(data_source)
print('current_prices: ', current_prices)

current_prices:  {'asset1': 0.8444218515250481, 'asset2': 0.7579544029403025}


In [234]:
class Portfolio:
    def __init__(self, assets: list, init_cash=1_000_000, acc_type="cash", init_margin=1.0):
        self.ledger = {asset: 0. for asset in assets}
        self.init_cash = init_cash
        self.type = acc_type
        
        self._cash = init_cash
        self.initial_margin = init_margin
        self._used_margin = 0.
        self._borrowed_cash = 0.
        self._available_margin = init_cash
        
    def __repr__(self):
        return f'cash: {self._cash} \nequity: {self.equity} \nledger: {str(self.ledger)}'
    
    @property
    def equity(self):
        return self._cash + sum((current_prices[k]*v for k, v in self.ledger.items())) - self._borrowed_cash
    
    @property
    def cash(self):
        return self._cash - self._borrowed_cash
    
    @property
    def purchasing_power(self):
        return self._cash / self.initial_margin

    def transaction_cash(self, asset_name, units):
        amount = current_prices[asset_name] * units
        self._cash -= amount
        self.ledger[asset_name] += units
    
    def transaction_margin(self, asset_name, units):
        amount = current_prices[asset_name] * units
        used_margin = amount * self.initial_margin
        self._borrowed_cash += amount * (1-self.initial_margin)
        self._cash -= used_margin
        self.ledger[asset_name] += units
        
    def order(self, asset_name: str, units: float):
        if self.type == "cash":
            self.transaction_cash(asset_name, units)
        elif self.type == "margin":
            self.transaction_margin(asset_name, units)
        return self

In [235]:
port = Portfolio(assets, acc_type="margin", init_margin=0.2)

In [241]:
port.order('asset1', 10000)

cash: 1000000.0 
equity: 1000000.0 
ledger: {'asset1': 0.0, 'asset2': 0.0}